In [1]:
import os

os.chdir('/home/enis/projects/nna/src/nna/exp/megan/run-1/')


In [2]:
# import run
# import nna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchaudio
torchaudio.set_audio_backend("sox_io")

import numpy as np



/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
torch.manual_seed(94)

np.random.seed(94)

torch.backends.cudnn.benchmark = False

In [4]:
best_solution_per_taxo_w_loc_name={'1.0.0': [['44', '46', '17', '14'],
  ['11', '34', '27'],
  ['31', '50', '18', '12', '30', '39', '48', '45']],
 '3.0.0': [['40', '20', '14', '17', '13', '36', '25', '33'],
  ['18', '38', '39'],
  ['32', '45']],
 'X.X.X': [['45', '14', '27', '25', '34', '46', '29', '18', '38'],
  ['36'],
  ['32', '20', '21']],
 '1.1.10': [['49', '48', '19', '16', '22', '37', '29', '25', '31', '27'],
  ['46', '20', '11', '33', '24'],
  ['17', '21', '39', '30', '38', '18', '47', '50', '14']],
 '1.1.0': [['12',
   '37',
   '11',
   '22',
   '18',
   '44',
   '29',
   '46',
   '13',
   '34',
   '25',
   '24',
   '17',
   '40',
   '31',
   '27',
   '14'],
  ['19', '16', '39', '30', '38', '41'],
  ['50', '20', '47', '49', '48', '21', '15', '36']],
 '1.3.0': [['21', '40', '32', '39', '41', '44'],
  ['50', '20', '38', '11'],
  ['24', '19', '27', '14']],
 '1.1.8': [['20', '16', '11', '37', '15', '25', '31'],
  ['21', '49', '38'],
  ['22', '40']],
 '1.1.7': [['16', '30', '15', '46', '27'],
  ['49', '25', '24'],
  ['20', '39', '48', '19', '38', '22', '29', '11', '40']],
 '0.2.0': [['17'], ['24', '27'], ['19', '15']],
 '0.0.0': [['29'], ['24'], ['12', '27']],
 '1.1.3': [['46', '25'], ['24'], ['27', '22', '38']],
 '2.1.0': [['34', '33'], ['36'], ['46']],
 '0.1.0': [['29'], ['25'], ['24']]}
error_because_small = ['1.2.4',
'1.1.5',
'1.2.0',
'0.4.0']


In [5]:
import runconfigs
import wandb
from ignite.metrics import Accuracy, Loss
from ignite.contrib.metrics import ROC_AUC

In [6]:
# from nna.exp import augmentations,
from nna.exp import modelArchs,runutils

In [7]:
# wandb.init(config=runconfigs.default_config, project=runconfigs.PROJECT_NAME)
# config = wandb.config
config = runconfigs.default_config
# wandb.config.update(args) # adds all of the arguments as config variables

params = {
    'batch_size': config['batch_size'],
    'shuffle': True,
    'num_workers': 0
}


In [8]:
device = torch.device(
    f"cuda:{config['device']}" if torch.cuda.is_available() else "cpu")


In [9]:
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split

# labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# sourcePath = Path("/scratch/enis/data/nna/labeling/splits/")


In [10]:
CATEGORY_COUNT = 2


In [11]:
from collections import Counter


In [12]:
 # RAW DATA
def load_raw_data(sample_count,CATEGORY_COUNT):
    sample_count = 1200
    X = np.empty((sample_count,480000),dtype=np.float32)
    y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
    return X_train,X_test,X_val,y_train,y_test,y_val

def mock_raw_data(sample_count,CATEGORY_COUNT):
#     sample_count = 30
    X = np.empty((sample_count*CATEGORY_COUNT,480000),dtype=np.float32)
    # y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    y_true=[]
    for i in range(CATEGORY_COUNT):
        y_true.extend([i]*sample_count)
    print(len(y_true))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (0 , 3400))
#     X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
#     print(y_train)
    return X_train,X_test,X_val,y_train,y_test,y_val
    


In [13]:
## Load real data rather than mock 
    # MVP1: delete parts longer than 10 seconds
import run
audio_dataset,_ = run.prepare_dataset()

output_file_path = '/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/files_as_np_filtered_v3_int16.pkl'
audio_dataset.load_audio_files(output_file_path)
audio_dataset.pick_channel_by_clipping()


/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['seabirds'] 1.0
['canids'] 1.0
['other-flare'] 11.0
['other-rain'] 20.0

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-aircraft'] 107.0
['loons'] 29.0
['other-car'] 37.0
('-> 102 number of samples are deleted because their taxonomy category does '
 'not have enough data.')
-> classes that do not have

In [14]:
sound_ins=next(iter(audio_dataset.items()))

In [15]:
sound_ins[1].location_id

'11'

In [16]:
# sound_ins[1].taxo_code
# classA = 1.1.7 #'duck-goose-swan']
# classB = 0.2.0 # other-aircraft
sampleTest= []
y=[]
location_id_info = []
expected_len=10
for sound_ins in audio_dataset.values():
    if sound_ins.taxo_code in ['1.1.10','1.1.7']:
        y.append(sound_ins.taxo_code)
        location_id_info.append(sound_ins.location_id)
        if sound_ins.length<10:
            tile_reps = (expected_len/(sound_ins.length)+1)
            repeated_data = np.tile(sound_ins.data,int(tile_reps))
            repeated_data = repeated_data[:expected_len*sound_ins.sr]
            sampleTest.append(repeated_data)
        else:
            sampleTest.append(sound_ins.data[:expected_len*sound_ins.sr])

len(sampleTest),len(y)

# sampleTest=np.array(sampleTest)

(589, 589)

In [17]:
from numpy import argmax
# define input string
# define universe of possible input values
alphabet = ['1.1.10','1.1.7']
# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# integer encode input data
integer_encoded = [char_to_int[char] for char in y]
# print(integer_encoded)
# one hot encode
onehot_encoded = list()
for value in integer_encoded:
	letter = [0 for _ in range(len(alphabet))]
	letter[value] = 1
	onehot_encoded.append(letter)
# print(onehot_encoded)
# invert encoding
inverted = int_to_char[argmax(onehot_encoded[0])]
# print(inverted)
onehot_encoded=np.array(onehot_encoded)

In [18]:
X_train,X_test, X_val, y_train, y_test,y_val  = [],[],[],[],[],[]
for sample,taxo,y_val_ins,loc_id in  zip(sampleTest,y,onehot_encoded,location_id_info):
    if loc_id in best_solution_per_taxo_w_loc_name[taxo][0]:
        X_train.append(sample)
        y_train.append(y_val_ins)
    elif loc_id in best_solution_per_taxo_w_loc_name[taxo][1]:
        X_test.append(sample)
        y_test.append(y_val_ins)
    elif loc_id in best_solution_per_taxo_w_loc_name[taxo][2]:
        X_val.append(sample)
        y_val.append(y_val_ins)
    else:
        print('error')


In [19]:
len(sampleTest),361+113+115

(589, 589)

In [20]:
X_train,X_test,X_val=np.array(X_train),np.array(X_test),np.array(X_val)
y_train,y_test,y_val=np.array(y_train),np.array(y_test),np.array(y_val)


In [21]:
# X_train, X_test, y_train, y_test = train_test_split(
#                 sampleTest, onehot_encoded, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#                 X_train, y_train, test_size=0.25,random_state=42)

In [22]:
X_train,X_test,X_val=torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(),torch.from_numpy(X_val).float()
y_train,y_test,y_val=torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float(),torch.from_numpy(y_val).float()


In [23]:
X_train.shape,X_test.shape,X_val.shape

(torch.Size([361, 480000]),
 torch.Size([113, 480000]),
 torch.Size([115, 480000]))

In [24]:
y_train.shape,y_test.shape,y_val.shape

(torch.Size([361, 2]), torch.Size([113, 2]), torch.Size([115, 2]))

In [25]:
y_val[0:3]

tensor([[0., 1.],
        [0., 1.],
        [1., 0.]])

In [26]:
# # X_train,X_test,X_val,y_train,y_test,y_val = load_raw_data('labelsbyhumanpath','sourcePath')
# X_train,X_test,X_val,y_train,y_test,y_val = mock_raw_data(4,CATEGORY_COUNT)
# X_train,X_test,X_val=torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(),torch.from_numpy(X_val).float()
# y_train,y_test,y_val=torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float(),torch.from_numpy(y_val).float()

# # labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# # with open(labelsbyhumanpath/"np_array_Ymatrix.npy", 'rb') as f:
# #     y_true = np.load(f)

# Counter(np.argmax(y_train,axis=1).tolist()),Counter(np.argmax(y_val,axis=1).tolist()),Counter(np.argmax(y_test,axis=1).tolist())


In [27]:

# X_train2=np.interp(X_train, (X_train.min(), X_train.max()), (-32768 , 32767))
# torch.from_numpy(X_train2).float()


In [28]:
# y_val

In [29]:
# smaple_index=10
# X_train[smaple_index,:],y_train[smaple_index,]


In [30]:
# X_train[0,:].shape

In [31]:
# multiply(torch.ones((1,2)),torch.ones((1,2)))

In [32]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self, maxMelLen, sampling_rate):
        # sr = 44100 etc
        self.maxMelLen = maxMelLen
        self.sampling_rate = sampling_rate

    def __call__(self, sample):
        x, y = sample
        #https://github.com/PCerles/audio/blob/3803d0b27a4e13efa760227ef6c71d0f3753aa98/test/test_transforms.py#L262
        #librosa defaults
        n_fft = 2048
        hop_length = 512
        power = 2.0
        n_mels = 128
        n_mfcc = 40
        # htk is false in librosa, no setting in torchaudio -?
        # norm is 1 in librosa, no setting in torchaudio -?
        melspect_transform = torchaudio.transforms.MelSpectrogram(sample_rate=self.sampling_rate, window_fn=torch.hann_window,
                                                                  hop_length=hop_length, n_mels=n_mels, n_fft=n_fft)

    
        db_transform = torchaudio.transforms.AmplitudeToDB("power", 80.)
        mel = melspect_transform(x.reshape(-1))
        an_x = db_transform(mel)
        #librosa version
#         mel = librosa.feature.melspectrogram(y=x.reshape(-1),
#                                              sr=self.sampling_rate)
#         an_x = librosa.power_to_db(mel, ref=np.max)
#         an_x = an_x.astype("float32")
#         y = y.astype('float32')
#         print(an_x.shape)
        an_x = an_x[:, :self.maxMelLen]
        # 2-d conv
#         x = an_x.reshape(1, *an_x.shape[:])
        # 1-d conv
        x = an_x.reshape(1, an_x.shape[0]*an_x.shape[1])

        
        return x,y

# #test
# maxMelLen_test = 850
# SAMPLING_RATE_test = 48000
# sample_len_seconds = 10
# # to_tensor works on single sample
# sample_count = 1
# xx_test = torch.ones((sample_count,SAMPLING_RATE_test*sample_len_seconds))
# y_values = torch.ones(sample_count)
# 
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out,y_out=toTensor((xx_test,y_values))
# x_out.shape,y_out.shape

In [33]:
# X_train,X_test,X_val,y_train,y_test,y_val
X_train[:].shape

torch.Size([361, 480000])

In [34]:
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out2,y_out=toTensor((X_train[1:2,:],y_train))
# x_out.shape,y_out.shape

In [35]:
# torch.mean(x_out[0]),torch.mean(x_out2[0])
# y_train

In [36]:
# X_train,y_train

In [37]:

# pitch = augmentations.pitch_shift_n_stepsClass(
#     runconfigs.SAMPLING_RATE, config['pitch_shift_n_steps'])
# noise = augmentations.addNoiseClass(config['noise_factor'])
# strech = augmentations.time_stretchClass(runconfigs.SAMPLING_RATE*runconfigs.EXCERPT_LENGTH,
#                                             config['time_stretch_factor'],
#                                             isRandom=True)
# shift = augmentations.shiftClass(config['roll_rate'], isRandom=True)
maxMelLen = 938 # old 850
# toTensor = augmentations.ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)
toTensor = ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)

In [38]:
from ignite.metrics import EpochMetric


def roc_auc_perClass_compute_fn(y_preds, y_targets):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError(
            "This contrib module requires sklearn to be installed.")

    y_true = y_targets.numpy()
    y_pred = y_preds.numpy()
#     print(y_pred,y_true)
#     res = []
#     for y_true_perClass_Index in y_true.shape[1]:
#         res.append(
#             roc_auc_score(y_true[:, y_true_perClass_Index],
#                           y_pred[:, y_true_perClass_Index]))
    res = roc_auc_score(y_true, y_pred, average=None)
    return res


#[docs]
class ROC_AUC_perClass(EpochMetric):
    """Computes Area Under the Receiver Operating Characteristic Curve (ROC AUC)
  accumulating predictions and the ground-truth during an epoch and applying
  `sklearn.metrics.roc_auc_score <http://scikit-learn.org/stable/modules/generated/
  sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score>`_ .

  Args:
      output_transform (callable, optional): a callable that is used to transform the
          :class:`~ignite.engine.engine.Engine`'s ``process_function``'s output into the
          form expected by the metric. This can be useful if, for example, you have a multi-output model and
          you want to compute the metric with respect to one of the outputs.
      check_compute_fn (bool): Optional default False. If True, `roc_curve
          <http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#
          sklearn.metrics.roc_auc_score>`_ is run on the first batch of data to ensure there are
          no issues. User will be warned in case there are any issues computing the function.

  ROC_AUC expects y to be comprised of 0's and 1's. y_pred must either be probability estimates or confidence
  values. To apply an activation to y_pred, use output_transform as shown below:

  .. code-block:: python

      def activated_output_transform(output):
          y_pred, y = output
          y_pred = torch.sigmoid(y_pred)
          return y_pred, y

      roc_auc = ROC_AUC(activated_output_transform)

  """
    def __init__(self,
                 output_transform=lambda x: x,
                 check_compute_fn: bool = False):
#         print(output_transform)
        super(ROC_AUC_perClass,
              self).__init__(roc_auc_perClass_compute_fn,
                             output_transform=output_transform,
                             check_compute_fn=check_compute_fn)


In [39]:

def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    """
  Utility function for computing output of convolutions
  takes a tuple of (h,w) and returns a tuple of (h,w)
  """
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor(((h_w[0] + (2 * pad) - (dilation *
                                      (kernel_size[0] - 1)) - 1) / stride) + 1)
    w = floor(((h_w[1] + (2 * pad) - (dilation *
                                      (kernel_size[1] - 1)) - 1) / stride) + 1)
    return h, w

# mel.shape,an_x.shape,X_train.shape
class testModel(nn.Module):
    '''A simple model for testing by overfitting.
    '''
    def __init__(self, out_channels, h_w, kernel_size, FLAT=False,output_shape=(10,)):
        # h_w: height will be always one since we use 1d convolution 
        super(testModel, self).__init__()
        self.out_channels = out_channels
        #### CONV
        self.conv1 = nn.Conv1d(in_channels=1, # depth of image == depth of filters
                               out_channels=self.out_channels, # number of filters 
                               kernel_size=kernel_size, # size of the filters/kernels
                               padding=1)

        self.conv1_shape = conv_output_shape(h_w, kernel_size=kernel_size, stride=1, pad=1, dilation=1)
        # conv is 1d
        self.conv1_shape = (1,self.conv1_shape[1])
        
        self.fc1 = nn.Linear(self.out_channels * self.conv1_shape[0] *self.conv1_shape[1], 75)  # 100

        self.fc2 = nn.Linear(75,output_shape[0])
        
        
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

        
    def forward(self, x):
#         x = x.reshape(1,)
#         print(x.shape) #  50,1,108800 (850*128)
        x = F.relu(self.conv1(x))
#         x = self.pool(x)
        # x = self.drop(x)
#         print(x.shape)# 58, 2, 108801
#         print(self.conv1_shape)
#         print(x.shape)
        x = x.view(-1, self.out_channels * self.conv1_shape[0] *self.conv1_shape[1])
        # batch_norm is missing
        x = F.relu((self.fc1(x)))
        x = (self.fc2(x))

#         x = self.drop(x)

#         x = self.fc4(x)
#         x = torch.sigmoid(x)
#                 x = F.log_softmax(x,dim=1)
        return x

# # test
# input_shape=(1,(938*128))
# output_shape=(10,)
# testModel_ins=testModel(out_channels=2,h_w=input_shape,kernel_size=2,output_shape=output_shape)
# # a.conv1.weight
# a_out=testModel_ins(torch.ones((3,1,input_shape[1])))

# a_out_correct=torch.zeros(a_out.shape)
# a_out_correct[0][:]=1
# a_out_correct
# a_out.detach().numpy()

# torch.exp(a_out),a_out

In [40]:

# for y_true_perClass_Index in a_out_correct.shape[1]:
#     print(y_true_perClass_Index)

In [41]:
# from ignite.contrib.metrics import ROC_AUC
# from nna.exp.metrics import ROC_AUC_perClass
def activated_output_transform(output):
    y_pred, y = output
#     y_pred = torch.exp(y_pred)
    return y_pred, y

In [42]:
# asd=ROC_AUC_perClass(activated_output_transform)
# asd.update((a_out,a_out_correct))
# asd.compute()

In [43]:

transformCompose = transforms.Compose([
#     pitch,
#     strech,
#     shift,
#     noise,
    toTensor,
])


sound_datasets = {
    phase: runutils.audioDataset(XY[0], XY[1], transform=transformCompose)
    for phase, XY in
    zip(['train', 'val', 'test'],
        [[X_train, y_train], [X_val, y_val], [X_test, y_test]])
}

dataloaders = {
    x: torch.utils.data.DataLoader(sound_datasets[x], **params)
    for x in ['train', 'val', 'test']
}

# this will change
h_w = [128, 938]
kernel_size = (5, 5)
# if config['CNNLayer_count'] == 1:
#     model = modelArchs.NetCNN1(config['CNN_filters_1'], h_w,
#                                 kernel_size).float().to(device)

# if config['CNNLayer_count'] == 2:
#     model = modelArchs.NetCNN2(config['CNN_filters_1'], config.CNN_filters_2,
#                                 h_w, kernel_size,
#                                 kernel_size).float().to(device)

#simpler model

output_shape=(CATEGORY_COUNT,)
model = testModel(out_channels=2,h_w=(1,h_w[0]*h_w[1]),kernel_size=kernel_size[0]*kernel_size[0],output_shape=output_shape)
model.float().to(device)

# device is defined before

model.float().to(device)  # Move model before creating optimizer
optimizer = torch.optim.AdamW(model.parameters(),
#                                 weight_decay=config['weight_decay'],
                              amsgrad=True,
                             )



criterion = nn.BCEWithLogitsLoss()
# statHistory={'valLoss':[],'trainLoss':[],'trainAUC':[],'valAUC':[]}

metrics = {
    'loss': Loss(criterion),  # 'accuracy': Accuracy(),
#     'ROC_AUC': ROC_AUC(runutils.activated_output_transform),
    'ROC_AUC': ROC_AUC_perClass(activated_output_transform),

}


In [44]:
model

testModel(
  (conv1): Conv1d(1, 2, kernel_size=(25,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=240084, out_features=75, bias=True)
  (fc2): Linear(in_features=75, out_features=2, bias=True)
)

In [45]:
model.conv1.weight

Parameter containing:
tensor([[[-0.1655, -0.0348,  0.0037, -0.0861,  0.2388, -0.2812, -0.0201,
           0.1076, -0.1862, -0.2773,  0.0596,  0.0522, -0.1000, -0.2006,
          -0.1242, -0.0324,  0.0544, -0.0058,  0.0053,  0.0320,  0.2231,
           0.0766, -0.2167,  0.2076,  0.1998]],

        [[ 0.0346,  0.0648, -0.1567,  0.2104, -0.1485,  0.1780, -0.0413,
           0.0800,  0.1863,  0.2529,  0.1293, -0.0186, -0.0285, -0.0969,
           0.0148, -0.2024, -0.1476, -0.2799, -0.2271,  0.0679,  0.2659,
           0.1187, -0.1989,  0.1877,  0.0064]]], device='cuda:1',
       requires_grad=True)

In [46]:
model.conv1

Conv1d(1, 2, kernel_size=(25,), stride=(1,), padding=(1,))

In [47]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0007, -0.0033, -0.0009,  ..., -0.0034, -0.0042,  0.0023],
        [-0.0037,  0.0028,  0.0015,  ..., -0.0014, -0.0025, -0.0016],
        [-0.0012,  0.0021,  0.0001,  ...,  0.0049,  0.0011, -0.0025],
        ...,
        [ 0.0035, -0.0036, -0.0008,  ..., -0.0031,  0.0011,  0.0029],
        [-0.0037,  0.0005,  0.0012,  ..., -0.0047,  0.0044, -0.0038],
        [ 0.0009,  0.0040,  0.0013,  ...,  0.0014, -0.0030,  0.0012]],
       device='cuda:1', requires_grad=True)

In [48]:
model.fc2.weight

Parameter containing:
tensor([[ 0.0407, -0.0883, -0.0618,  0.1564,  0.1131, -0.0413, -0.1120, -0.1032,
          0.2342, -0.2008, -0.0285,  0.1741, -0.1752,  0.0395,  0.2085, -0.0790,
          0.1583,  0.2439, -0.0177,  0.2277,  0.1519, -0.0567, -0.1576,  0.1256,
          0.1472, -0.2423,  0.0648, -0.0325,  0.1111,  0.0138,  0.2674,  0.1202,
         -0.1830, -0.0928, -0.1099, -0.1313,  0.1599, -0.1447,  0.1281, -0.2193,
          0.0106, -0.2092, -0.1733, -0.0967, -0.0441,  0.0586, -0.0745, -0.2231,
          0.0274, -0.2694, -0.1276, -0.0869, -0.0896,  0.2333, -0.0148, -0.0837,
          0.0109,  0.2598,  0.1989, -0.2720, -0.0190, -0.1238, -0.1834, -0.2188,
         -0.1620,  0.1346,  0.1273, -0.0995, -0.0975, -0.0159,  0.1240, -0.2533,
          0.0304, -0.1887, -0.2329],
        [ 0.2224, -0.0156, -0.1965,  0.0838,  0.0621, -0.1233,  0.1867,  0.2192,
          0.1927, -0.0953,  0.0352,  0.0057, -0.0354, -0.2657, -0.1950, -0.0191,
         -0.0105, -0.0878,  0.2389,  0.1291,  0.13

In [49]:
model.fc2.weight[:,35:60]

tensor([[-0.1313,  0.1599, -0.1447,  0.1281, -0.2193,  0.0106, -0.2092, -0.1733,
         -0.0967, -0.0441,  0.0586, -0.0745, -0.2231,  0.0274, -0.2694, -0.1276,
         -0.0869, -0.0896,  0.2333, -0.0148, -0.0837,  0.0109,  0.2598,  0.1989,
         -0.2720],
        [ 0.0400,  0.0635, -0.1201,  0.0607, -0.0196, -0.0518,  0.0960, -0.2407,
          0.0508, -0.1574,  0.1611, -0.2292,  0.0912, -0.2528, -0.2286,  0.0468,
          0.2775,  0.1522,  0.1719,  0.1319,  0.1590,  0.0493, -0.2257,  0.2738,
         -0.1708]], device='cuda:1', grad_fn=<SliceBackward>)

In [50]:

for m in (torch.sum(model.conv1.weight),torch.sum(model.fc1.weight),torch.sum(model.fc2.weight),
torch.mean(model.conv1.weight),torch.mean(model.fc1.weight),torch.mean(model.fc2.weight)):
    print(m.item())


-0.2189263105392456
-10.042251586914062
-1.663476586341858
-0.004378526005893946
-5.577076649387891e-07
-0.01108984462916851


In [ ]:
#amsgrad=True, init.xavier_uniform_, 25,75

print('ready ?')
runutils.run(model, dataloaders, optimizer, criterion, metrics, device,config, runconfigs.PROJECT_NAME)


In [52]:
from ignite.engine import create_supervised_evaluator
import glob

In [53]:
# test_loader = dataloaders['test']
# test_evaluator = create_supervised_evaluator(model,
#                                               metrics=metrics,
#                                               device=device)



In [54]:
model_files_path = '/home/enis/projects/nna/src/nna/exp/megan/run-1/wandb/run-20210114_155758-x6ltobzt/files/'
model_path= model_files_path + 'best_model_9_ROC_AUC=0.8751.pt'
model_list = glob.glob(model_files_path+'best_model*')
model_list.sort()
for model_path in model_list:
    print(model_path)
    model_saved = testModel(out_channels=2,h_w=(1,h_w[0]*h_w[1]),kernel_size=kernel_size[0]*kernel_size[0],output_shape=output_shape)
    model_saved.load_state_dict(torch.load(model_path))
    model_saved.eval().to(device)
    test_loader = dataloaders['test']
    test_evaluator = create_supervised_evaluator(model_saved,
                                                  metrics=metrics,
                                                  device=device)

    test_evaluator.run(test_loader)
    roc_auc_array_test = test_evaluator.state.metrics['ROC_AUC']

    print('test loss', test_evaluator.state.metrics['loss'])
    print('test roc auc', roc_auc_array_test, )
    break

# BEST
# /home/enis/projects/nna/src/nna/exp/megan/run-1/wandb/run-20210114_155758-x6ltobzt/files/best_model_10_ROC_AUC=0.8853.pt
# test loss 1.0067617787724048
# test roc auc [0.84182099 0.85300926]

/home/enis/projects/nna/src/nna/exp/megan/run-1/wandb/run-20210114_155758-x6ltobzt/files/best_model_10_ROC_AUC=0.8853.pt


/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)


test loss 1.0067618106846261
test roc auc [0.84182099 0.85300926]


In [58]:
model_saved

In [58]:
outputs=[]
labelss=[]
for inputs, labels in dataloaders['test']:
    inputs = inputs.float().to(device)
    output = model_saved(inputs)
    output = output.to("cpu")
    index = output.data.numpy()
    outputs.append(index)
    labelss.append(labels)
#     print(index)

In [92]:
t=0
f=0
limit=0
for m,n in zip(outputs[0]>limit,labelss[0]>0):
    if (m==n.numpy()).all():
        t+=1
    else:
        f+=1
t,f

(41, 17)

In [73]:
labelss[0]>0,

tensor([[ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [ True, False],
        [ True, False],
        [ True, False],
        [ True, False],
        [ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [ True, False],
        [ True, False],
        [ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [False,  True],
        [False,  True],
        [ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [ True, False],
        [False,  True],
        [ True, False],
        [ True, False],
        [ True, False],
        [ True, False],
        [False, 

In [63]:
for m,n in zip(outputs[0],labelss[0]):
    print(m,n)

[ 2.4041343 -3.5336268] tensor([1., 0.])
[-4.721764   3.9302242] tensor([0., 1.])
[-0.6255104  0.5269026] tensor([1., 0.])
[ 3.0626864 -6.2202144] tensor([1., 0.])
[-1.5258244  1.369283 ] tensor([1., 0.])
[-1.1834712  1.7585473] tensor([0., 1.])
[ 0.5676827 -0.9845088] tensor([1., 0.])
[-0.23613587 -0.24668674] tensor([1., 0.])
[-8.427179  8.767305] tensor([1., 0.])
[ 31.213781 -31.929234] tensor([0., 1.])
[ 2.3647826 -3.863914 ] tensor([1., 0.])
[ 14.386643 -11.592027] tensor([1., 0.])
[ 42.059864 -42.190323] tensor([1., 0.])
[ 15.248289 -18.514023] tensor([1., 0.])
[ 2.518043 -2.029641] tensor([1., 0.])
[-0.2652089  0.6528234] tensor([1., 0.])
[-3.255232   3.6832774] tensor([1., 0.])
[-0.23310822  0.35905677] tensor([0., 1.])
[ 4.0603647 -5.3932114] tensor([1., 0.])
[ 2.0881953 -2.0108125] tensor([1., 0.])
[-0.624428   -0.87871474] tensor([1., 0.])
[ 7.186704  -7.6504793] tensor([1., 0.])
[ 5.342203  -7.0520372] tensor([1., 0.])
[-1.6651335  0.6895987] tensor([0., 1.])
[-2.3494873   

test loss 0.575299070472211
test roc auc [0.73958333 0.73533951]


In [51]:
model.conv1.weight

Parameter containing:
tensor([[[-0.1916, -0.1012, -0.0548, -0.1832,  0.0159, -0.1202,  0.2493,
           0.1585,  0.2155, -0.2738, -0.0770,  0.1383, -0.2549,  0.0978,
           0.2230,  0.0194, -0.0346,  0.0620, -0.2393, -0.0458, -0.1269,
          -0.1478, -0.1659, -0.0260, -0.0971]],

        [[-0.1530,  0.1615, -0.0582, -0.2440, -0.1321,  0.1978, -0.0892,
          -0.2765,  0.0416, -0.2716,  0.1296, -0.1338,  0.0496,  0.2502,
          -0.1181,  0.2220,  0.2381, -0.1465, -0.0384, -0.1267,  0.0831,
          -0.1753, -0.2311, -0.2757,  0.2426]]], device='cuda:1',
       requires_grad=True)

In [55]:
for m in (torch.sum(model.conv1.weight),torch.sum(model.fc1.weight),torch.sum(model.fc2.weight),
torch.mean(model.conv1.weight),torch.mean(model.fc1.weight),torch.mean(model.fc2.weight)):
    print(m.item())


-0.28379660844802856
-35989.8203125
-1.550400972366333
-0.005675931926816702
-0.0019987348932772875
-0.010336006991565228


In [52]:
for m in (torch.sum(model.conv1.weight),torch.sum(model.fc1.weight),torch.sum(model.fc2.weight),
torch.mean(model.conv1.weight),torch.mean(model.fc1.weight),torch.mean(model.fc2.weight)):
    print(m.item())


-1.8156369924545288
-2.30564022064209
1.113730788230896
-0.03631274029612541
-1.2804630955542962e-07
0.007424871902912855


In [135]:
model.conv1.weight

Parameter containing:
tensor([[[-0.2487, -0.1000, -0.2389, -0.0892,  0.0787,  0.2289,  0.0016,
           0.1942,  0.2148,  0.0233, -0.2194, -0.1379,  0.2334, -0.2496,
           0.1046,  0.0043,  0.1701, -0.0154,  0.2012,  0.0757, -0.2177,
           0.1922, -0.1940,  0.0119, -0.2180]],

        [[-0.1250, -0.1686, -0.1103, -0.0830, -0.0704,  0.2523, -0.0615,
          -0.0895, -0.0651, -0.2415, -0.0692, -0.0974, -0.2820,  0.0661,
           0.1650,  0.1759,  0.2147, -0.2037,  0.1802,  0.1424, -0.0963,
           0.2779,  0.1434, -0.2229,  0.2743]]], device='cuda:1',
       requires_grad=True)

In [136]:
model.fc1.weight

Parameter containing:
tensor([[-6.3913e-03, -3.9652e-04,  1.5468e-03,  ..., -3.7154e-03,
         -4.6094e-03,  4.2543e-03],
        [-5.4962e-03, -3.2947e-03, -9.8121e-03,  ...,  4.6668e-03,
         -1.4585e-03,  1.4293e-03],
        [-1.3380e-03, -8.9816e-03, -1.0979e-02,  ...,  1.5270e-03,
         -4.9358e-03,  1.2266e-03],
        ...,
        [-5.4898e-03, -9.9627e-03, -5.8345e-03,  ...,  3.9291e-03,
         -3.1578e-03, -2.2261e-03],
        [ 3.1439e-02, -3.5185e-03, -9.0754e-03,  ...,  6.5032e-05,
         -3.0601e-03,  3.2819e-04],
        [-6.9045e-03,  1.5367e-03, -7.8508e-03,  ..., -3.6295e-03,
         -1.8707e-03,  2.7540e-03]], device='cuda:1', requires_grad=True)

In [137]:
model.fc2.weight

Parameter containing:
tensor([[-0.0258,  0.1852,  0.2271,  0.1194,  0.1158,  0.0520, -0.2124,  0.2611,
          0.1001,  0.2018, -0.0867, -0.0450, -0.2670, -0.0627,  0.1312,  0.1560,
         -0.1971,  0.2356, -0.2341,  0.2606,  0.0816, -0.1185,  0.1474,  0.2373,
         -0.1709, -0.1620,  0.2559, -0.1174, -0.2700, -0.3141,  0.1292,  0.1363,
          0.2860,  0.1132, -0.0830, -0.0842,  0.0821,  0.1297,  0.1850,  0.1338,
          0.1675, -0.2371, -0.1067, -0.1755,  0.1127,  0.1066, -0.0880,  0.2650,
         -0.2147, -0.0141,  0.1902, -0.0046,  0.0631,  0.0874, -0.0506, -0.2295,
         -0.1659,  0.0087,  0.2414,  0.0652,  0.1240,  0.2610,  0.2491,  0.0604,
         -0.2192,  0.0183,  0.1401,  0.0713, -0.1225,  0.0926, -0.0011, -0.1932,
         -0.0828,  0.0607,  0.2029],
        [ 0.1402, -0.0403, -0.1288,  0.1159, -0.2280, -0.0907,  0.1757, -0.0119,
          0.1990, -0.0768,  0.0439, -0.0339,  0.1010,  0.1045, -0.2024,  0.2035,
         -0.2426,  0.0380,  0.2121,  0.2308,  0.18

In [139]:
model

testModel(
  (conv1): Conv1d(1, 2, kernel_size=(25,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=240084, out_features=75, bias=True)
  (fc2): Linear(in_features=75, out_features=2, bias=True)
)

In [ ]:

example_input = (torch.ones((1,1,h_w[0]*h_w[1]))*-32767)
example_input = example_input.float().to(device)
out=model(example_input)
out

activated_output_transform((out,out))

In [ ]:
torch.Tensor([10,10,10])

In [ ]:
sum(model.fc1.weight)

In [ ]:
m = nn.Sigmoid()

loss = nn.BCEWithLogitsLoss()
input = torch.tensor([100.0,100,100], requires_grad=True)
# target = torch.empty(3).random_(2)
target = torch.ones(3)
output = loss((input), target)
input,target,output

In [ ]:
input

In [ ]:
target

In [ ]:
output

In [ ]:
m(input)